In [1]:
from profit.config import Config
from profit.run import Runner
from profit.run.test import MockupWorker

import logging
logging.basicConfig(level=logging.INFO)

In [2]:
base_config = Config.from_file('numpy.yaml')

# Internal Interface

In [3]:
from profit.run.test import InternalInterface, InternalRunnerInterface
wif = InternalInterface({}, 1)
rif = InternalRunnerInterface({}, 5, base_config['input'], base_config['output'])
wif.connect(rif)

print(rif.input_vars)
print(rif.output_vars)

[('u', 'float64'), ('v', 'float64')]
[('f', 'float64', ())]


# Memmap Interface

In [4]:
from profit.run.default import MemmapInterface, MemmapRunnerInterface
c = {}
MemmapRunnerInterface.handle_config(c, base_config)
print(c)
rif = MemmapRunnerInterface(c, 5, base_config['input'], base_config['output'])
rif.input[['u', 'v']][1] = 1, -1
wif = MemmapInterface(c, 1)
wif.output['f'] = 2 * wif.input['u']
wif.done()
print(rif.input)
print(rif.output)

{'path': '/data/Data/Uni/BA/profit/tests/run/interface.npy'}
[(0.,  0.) (1., -1.) (0.,  0.) (0.,  0.) (0.,  0.)]
[(0.,) (2.,) (0.,) (0.,) (0.,)]


# ZeroMQ Interface

In [3]:
from threading import Thread
from time import sleep
from profit.run.zeromq import ZeroMQInterface, ZeroMQRunnerInterface
c = {}
ZeroMQRunnerInterface.handle_config(c, base_config)
print(c)

def runner():
    rif = ZeroMQRunnerInterface(c, 5, base_config['input'], base_config['output'])
    rif.input[['u', 'v']][1] = 1, -1
    for i in range(3):
        rif.poll()
        sleep(0.5)
        print(f'runner {i}')
    print(f'output {rif.output}')
    print(f'internal {rif.internal}')
    rif.clean()

def worker():
    wif = ZeroMQInterface(c, 1)
    print(f'input {wif.input}')
    wif.output['f'] = 2 * wif.input['u']
    wif.time = 2
    wif.done()

rt = Thread(target=runner)
wt = Thread(target=worker)

rt.start()
wt.start()
wt.join()
rt.join()

INFO:Runner Interface:connected
INFO:Interface:connected
INFO:Interface:READY: received input data


{'bind': 'tcp://*:9000', 'connect': 'tcp://localhost:9000', 'timeout': 2500, 'retries': 3, 'retry-sleep': 1}
input (1., -1.)
runner 0
runner 1
runner 2
output [(0.,) (2.,) (0.,) (0.,) (0.,)]
internal [(False, 0,  0) ( True, 2, 10) (False, 0,  0) (False, 0,  0)
 (False, 0,  0)]
